In [1]:
!pip install torch-2.5.0a0+872d972e41.nv24.08.17622132-cp310-cp310-linux_aarch64.whl

Processing ./torch-2.5.0a0+872d972e41.nv24.08.17622132-cp310-cp310-linux_aarch64.whl
torch is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [2]:
!pip install numpy==1.26.1

In [3]:
!pip install transformers Pillow 'accelerate>=0.26.0' opencv-python

In [3]:
import requests
from PIL import Image
import torch
from transformers import AutoProcessor, LlavaOnevisionForConditionalGeneration
import cv2
import os

# Set CUDA environment and device
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

/home/jemar/Downloads/jace-llava-onevision/jace/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda


In [4]:
# Load model and processor
model_id = "llava-hf/llava-onevision-qwen2-0.5b-ov-hf"
model = LlavaOnevisionForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(0)
processor = AutoProcessor.from_pretrained(model_id)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [5]:
# Log model parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}")

Total Parameters: 893,675,552
Trainable Parameters: 893,675,552


In [6]:
# Check CUDA memory usage after model loading
print(f"Memory allocated (MB): {torch.cuda.memory_allocated(0) / (1024 ** 2):.2f}")
print(f"Max memory allocated (MB): {torch.cuda.max_memory_allocated(0) / (1024 ** 2):.2f}")


Memory allocated (MB): 1723.32
Max memory allocated (MB): 1723.32


In [15]:
# Function to capture an image from the webcam
def capture_image_from_webcam():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not access the webcam.")
        return None
    
    print("Press 's' to capture an image and 'q' to quit.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture image from webcam.")
            break

        cv2.imshow("Webcam Feed", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('s'):  # Save the image on 's' key press
            cap.release()
            cv2.destroyAllWindows()
            return Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        elif key == ord('q'):  # Quit on 'q' key press
            break

    cap.release()
    cv2.destroyAllWindows()
    return None

# Capture an image from the webcam
print("Opening webcam to capture images...")
webcam_image = capture_image_from_webcam()

Opening webcam to capture images...
Press 's' to capture an image and 'q' to quit.


In [16]:
if webcam_image is not None:
    print("Image captured successfully.")
    # Define a chat history and use `apply_chat_template` to get correctly formatted prompt
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What is this?"},
                {"type": "image"},
            ],
        },
    ]
    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

    # Prepare inputs
    print("Processing the captured image...")
    inputs = processor(images=webcam_image, text=prompt, return_tensors="pt").to(0, torch.float16)

    # Check CUDA memory usage after inputs are prepared
    print(f"Memory allocated after inputs (MB): {torch.cuda.memory_allocated(0) / (1024 ** 2):.2f}")
    print(f"Max memory allocated after inputs (MB): {torch.cuda.max_memory_allocated(0) / (1024 ** 2):.2f}")

    # Generate output
    print("Generating outputs...")
    output = model.generate(**inputs, max_new_tokens=200, do_sample=False)

    # Check CUDA memory usage after inference
    print(f"Memory allocated after inference (MB): {torch.cuda.memory_allocated(0) / (1024 ** 2):.2f}")
    print(f"Max memory allocated after inference (MB): {torch.cuda.max_memory_allocated(0) / (1024 ** 2):.2f}")

    # Decode and print result
    print("Decoding output...")
    print(processor.decode(output[0][2:], skip_special_tokens=True))
else:
    print("No image was captured. Exiting.")


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Image captured successfully.
Processing the captured image...
Memory allocated after inputs (MB): 1735.74
Max memory allocated after inputs (MB): 2029.29
Generating outputs...
Memory allocated after inference (MB): 1735.74
Max memory allocated after inference (MB): 2029.31
Decoding output...
 
What is this?assistant
The image shows a person wearing a lanyard with a badge around their neck. The badge appears to have text on it, but the specific details are not clear due to the angle and resolution of the photo. The person is also wearing a white t-shirt with a graphic design on it. The setting seems to be an indoor environment, possibly an office or a public space, as there are other people and objects in the background.
